# Regularización L2

In [ ]:
# @title
import numpy as np
import math
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display, clear_output

# Parámetros fijos de la Gaussiana
MU = 0.0
SIGMA = 1.0

# Umbral fijo para definir "colas" dentro del rango visible: colas si |w-μ| >= T·σ
T = 2.0  # cambia a 1.0 o 3.0 si prefieres

def normal_pdf(x, mu=0.0, sigma=1.0):
    return (1.0 / (np.sqrt(2*np.pi) * sigma)) * np.exp(-0.5 * ((x - mu)/sigma)**2)

def normal_cdf(z):
    return 0.5 * (1.0 + math.erf(z / math.sqrt(2.0)))

# Slider: rango del eje X (en múltiplos de σ)
k_slider = widgets.FloatSlider(
    value=4.0, min=1.0, max=8.0, step=0.5,
    description="Rango ±k·σ", continuous_update=False
)

# Checkbox: incluir o no la curva de costo w^2
show_cost = widgets.Checkbox(value=True, description="Mostrar costo $w^2$")

out = widgets.Output()

def update(*_):
    with out:
        clear_output(wait=True)

        k = float(k_slider.value)

        # Eje X: μ ± kσ
        x = np.linspace(MU - k*SIGMA, MU + k*SIGMA, 1200)
        pdf = normal_pdf(x, MU, SIGMA)

        # Umbrales de colas dentro del rango visible
        left_thr = MU - T*SIGMA
        right_thr = MU + T*SIGMA

        plt.figure(figsize=(10, 4))
        plt.plot(x, pdf, label=r"PDF $\mathcal{N}(0,1)$")

        # Sombrear colas SOLO si k > T
        if k > T:
            plt.fill_between(x[x <= left_thr], pdf[x <= left_thr], alpha=0.25, label=f"Cola izquierda (≤ -{T}σ)")
            plt.fill_between(x[x >= right_thr], pdf[x >= right_thr], alpha=0.25, label=f"Cola derecha (≥ +{T}σ)")
            plt.axvline(left_thr, linestyle=":")
            plt.axvline(right_thr, linestyle=":")

            p_central = 2 * normal_cdf(T) - 1.0
            p_tails = 1.0 - p_central
            tails_msg = f"P(|X-μ| > {T}σ) = {p_tails:.4f}"
        else:
            tails_msg = f"(k ≤ {T}) No hay espacio suficiente para mostrar colas definidas por ±{T}σ."

        # Curva de costo opcional
        if show_cost.value:
            cost = x**2
            cost_scaled = (cost / (np.max(cost) + 1e-12)) * np.max(pdf)
            plt.plot(x, cost_scaled, linestyle="--", label=r"Costo $w^2$ (escalado)")

        plt.axvline(MU, linestyle=":", label="μ = 0")
        plt.title(f"Gaussiana y colas (colas definidas por |w-μ| ≥ {T}σ)")
        plt.xlabel("w")
        plt.ylabel("PDF (y costo escalado si se activa)")
        plt.legend()
        plt.tight_layout()
        plt.show()

        print(tails_msg)

k_slider.observe(update, names="value")
show_cost.observe(update, names="value")

display(widgets.HBox([k_slider, show_cost]), out)
update()

Output()

## 1. Modelo de regresión lineal
Sea un modelo lineal con dos variables:

$$
\hat{y} = w_1 x_1 + w_2 x_2
$$

La función de costo clásica de **Ordinary Least Squares (OLS)** es:

$$
J_{\text{MSE}}(\mathbf{w}) =
\frac{1}{n}\sum_{i=1}^{n}\left(\hat{y}_{i} - y_{i}\right)^2
$$


## 2. Distancia de Minkowski

La **distancia de Minkowski** es una familia de distancias que generaliza varias métricas conocidas, controladas por un parámetro $p$.

Su fórmula es:

$$
d_p(\mathbf{x}, \mathbf{y})
=
\left(
\sum_{i=1}^{n} |x_i - y_i|^p
\right)^{\tfrac{1}{p}},
\quad p \ge 1
$$

donde:
- $\mathbf{x}, \mathbf{y} \in \mathbb{R}^n$ son dos puntos,
- $p$ define cómo se penalizan las diferencias entre componentes.

Casos importantes:
- $p = 1$ → **distancia Manhattan (L1)**
- $p = 2$ → **distancia Euclidiana (L2)**

Minkowski unifica distintas nociones de distancia en una sola fórmula; cambiar $p$ cambia la geometría del espacio.

## 3. Agregando regularización L2 a la función del error
La **regularización L2 (Ridge)** añade una penalización sobre la magnitud de los pesos:

$$
J_{\text{Ridge}}(\mathbf{w}) =
\underbrace{\frac{1}{n}\sum_{i=1}^{n}\left(\hat{y}^{(i)} - y^{(i)}\right)^2}_{\text{error de ajuste}}
+
\underbrace{\lambda \left(w_1^2 + w_2^2\right)}_{\text{penalización L2}}
$$

donde:
- $\lambda \ge 0$ controla la intensidad de la regularización.
- Pesos grandes implican mayor penalización.


La **distancia de Minkowski de grado 2** es la **distancia euclidiana**:
$$
d_2(\mathbf{x}, \mathbf{y})
=
\sqrt{\sum_{i=1}^{n} (x_i - y_i)^2}
$$

Si fijamos $\mathbf{y} = \mathbf{0}$, obtenemos la **norma euclidiana**:

$$
\|\mathbf{x}\|_2 = d_2(\mathbf{x}, \mathbf{0})
$$


La regularización Ridge penaliza el **cuadrado de la norma euclidiana** de los pesos:

$$
\lambda \|\mathbf{w}\|_2^2
=
\lambda \sum_{j} w_j^2
$$

Esto significa que Ridge:

- mide **qué tan lejos está el vector de pesos del origen** usando Minkowski de grado 2,
- penaliza **fuertemente los pesos lejanos al origen**,
- favorece soluciones con **pesos pequeños**.


Interpretación geométrica

- La distancia Minkowski de grado 2 define **esferas** (bolas euclidianas).
- Ridge restringe la solución a estar **cerca del origen dentro de una esfera L2** porque una norma mide el tamaño de un solo vector. Y ese tamaño se define como su distancia al origen.
- Por eso Ridge **encoge** los coeficientes, pero no los lleva exactamente a cero.



## 4. La metáfora geométrica: la “cresta” (*ridge*)

*Ridge* significa *una cresta larga y elevada sobre una superficie*.

En mínimos cuadrados ordinarios (OLS), la superficie del error suele tener la forma de un **valle alargado y plano**.

La regularización L2 modifica la función de costo:

$$
J(\mathbf{w})
=
\|\mathbf{X}\mathbf{w} - \mathbf{y}\|_2^2
+
\lambda \|\mathbf{w}\|_2^2
$$

Geométricamente, el término $\lambda \|\mathbf{w}\|_2^2$ **eleva** el fondo del valle y el efecto es como **levantar una cresta** sobre el valle original.
- El **subíndice 2** en $\|\mathbf{w}\|_2$ indica que se utiliza la **norma $\ell_2$** (norma euclidiana).
- El **superíndice 2** indica que dicha norma se **eleva al cuadrado**.

En forma explícita:

$$
\lambda \|\mathbf{w}\|_2^2
=
\lambda \sum_j w_j^2
$$

Se usa el cuadrado para eliminar la raíz, simplificar el gradiente y mantener una penalización suave y convexa.

La regularización no cambia la forma global del valle, pero **añade una elevación** que evita soluciones degeneradas.



# Regularización L1


La **regularización L1** introduce una penalización basada en la **norma L1**, que es un caso particular de la **distancia de Minkowski**.

### 1. Distancia de Minkowski

La distancia de Minkowski entre dos vectores $\mathbf{x}, \mathbf{y} \in \mathbb{R}^d$ está definida como:

$$
d_p(\mathbf{x}, \mathbf{y}) =
\left( \sum_{j=1}^{d} |x_j - y_j|^p \right)^{1/p}, \quad p \ge 1
$$

Casos particulares importantes:
- $p = 2$: distancia Euclidiana (norma L2)
- $p = 1$: distancia Manhattan (norma L1)

Para regularización, se considera la distancia al **origen**, es decir, una norma.



### 2. Norma L1 como distancia de Minkowski

La **norma L1** de un vector de parámetros $\mathbf{w}$ es:

$$
\|\mathbf{w}\|_1 = \sum_{j=1}^{d} |w_j|
$$

Esto equivale a la distancia de Minkowski con $p = 1$ entre $\mathbf{w}$ y el vector cero.



### 3. Regularización L1 en regresión

En un problema de regresión lineal, la función de costo regularizada con L1 es:

$$
J(\mathbf{w}) =
\frac{1}{2n}\|\mathbf{X}\mathbf{w} - \mathbf{y}\|_2^2
\;+\;
\lambda \|\mathbf{w}\|_1
$$

donde:
- $\|\mathbf{X}\mathbf{w} - \mathbf{y}\|_2^2$ mide el error de ajuste,
- $\|\mathbf{w}\|_1$ mide la distancia Manhattan de $\mathbf{w}$ al origen,
- $\lambda > 0$ controla la intensidad de la regularización.



### 4. Propiedades clave de la regularización L1

**(a) Esparsidad**
- Produce coeficientes exactamente cero.
- Realiza selección de variables de forma implícita.

**(b) Relación con Minkowski**
- Cambiar $p$ en Minkowski cambia la geometría de la restricción:
  - $p=1$: esquinas → esparsidad.
  - $p=2$: superficie suave → coeficientes pequeños pero no nulos.



### 5. Comparación conceptual L1 vs L2

| Regularización | Norma (Minkowski) | Geometría | Efecto |
|---------------|-------------------|-----------|--------|
| L1 (Lasso) | $p=1$ | Rombo | Coeficientes exactos en cero |
| L2 (Ridge) | $p=2$ | Esfera | Coeficientes pequeños |
| Elastic Net | $p=1$ + $p=2$ | Mixta | Esparsidad + estabilidad |




# Elastic Net

La **regularización Elastic Net** combina las penalizaciones **L1** y **L2**, integrando dos geometrías inducidas por la distancia de Minkowski. Esto permite equilibrar **esparsidad**, **estabilidad numérica** y **manejo de variables correlacionadas**.


### 1. Recordatorio: Minkowski y normas

La distancia de Minkowski para $p \ge 1$ es:

$$
d_p(\mathbf{x}, \mathbf{y}) =
\left( \sum_{j=1}^{d} |x_j - y_j|^p \right)^{1/p}
$$

Al medir la distancia al origen obtenemos normas:
- $p = 1 \Rightarrow \|\cdot\|_1$ (Manhattan)
- $p = 2 \Rightarrow \|\cdot\|_2$ (Euclidiana)

Elastic Net **no corresponde a un único valor de $p$**, sino a una **combinación de dos normas de Minkowski**.


### 2. Función de costo de Elastic Net

En regresión lineal, Elastic Net se define como:

$$
J(\mathbf{w}) =
\frac{1}{2n}\|\mathbf{X}\mathbf{w} - \mathbf{y}\|_2^2
\;+\;
\lambda \Big(
\alpha \|\mathbf{w}\|_1
+
(1-\alpha)\|\mathbf{w}\|_2^2
\Big)
$$

donde:
- $\lambda > 0$ controla la regularización total,
- $\alpha \in [0,1]$ controla el balance L1–L2,
- $\alpha = 1$ → Lasso,
- $\alpha = 0$ → Ridge.

---

### 3. Interpretación geométrica (Minkowski híbrida)

Elastic Net impone la restricción:

$$
\alpha \|\mathbf{w}\|_1 + (1-\alpha)\|\mathbf{w}\|_2^2 \le c
$$


### 4. Por qué Elastic Net funciona mejor con variables correlacionadas

- **L1 puro (Lasso):** tiende a seleccionar **una sola** variable dentro de un grupo altamente correlacionado.
- **L2 (Ridge):** distribuye el peso entre todas, pero no elimina ninguna.
- **Elastic Net:** selecciona **grupos de variables correlacionadas**, asignándoles coeficientes similares.


### 5. Propiedades teóricas


**(a) Esparsidad controlada**
- Producida por el término L1.
- Ajustable mediante $\alpha$.

**(b) Estabilidad numérica**
- El término L2 evita soluciones inestables cuando:
  - $p \gg n$
  - $\mathbf{X}^\top \mathbf{X}$ es mal condicionada o singular.


### 6. Elastic Net como regularización Minkowski compuesta

Desde el punto de vista geométrico:

- L1 mide complejidad usando la **distancia Manhattan**,
- L2 mide complejidad usando la **distancia Euclidiana**,
- Elastic Net **combina ambas nociones de distancia** para definir qué significa un modelo “simple”.



### 7. Comparación resumida

| Método | Norma (Minkowski) | Geometría | Comportamiento |
|------|-------------------|-----------|----------------|
| Lasso | $p=1$ | Rombo | Esparsidad fuerte |
| Ridge | $p=2$ | Esfera | Estabilidad |
| Elastic Net | $p=1$ + $p=2$ | Híbrida | Esparsidad + agrupamiento |



> **Elastic Net redefine la noción de complejidad del modelo como una combinación de distancias de Minkowski, logrando soluciones esparsas pero geométricamente más estables y realistas en presencia de correlación.**

Elastic Net suele superar a Lasso y Ridge en escenarios de alta dimensionalidad.